<a href="https://colab.research.google.com/github/saminjamshidi/Question_generation/blob/master/use_encode_as_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
!pip install nltk
!pip install transformers
!pip install torch-geometric
!python -m spacy download en_core_web_md

2023-11-20 19:52:50.346160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 19:52:50.346301: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 19:52:50.346431: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 19:52:52.089705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [132]:
import nltk
import torch
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
import spacy
# import networkx as nx
# import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [133]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
from torch_geometric.data import Data  # Add this import
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split


In [134]:
batch_size = 10
num_epochs = 10
query_max_size=0

In [135]:
import spacy
def word_embedding(unique_nodes):
# Load the spaCy model with GloVe vectors
  nlp = spacy.load("en_core_web_md")

  # Process each word in the list using spaCy and retrieve their GloVe word vectors
  word_embeddings = [nlp(word).vector for word in unique_nodes]
  return torch.tensor(word_embeddings)
# Now, 'word_embeddings' contains a list of word embeddings for each word in the word list



In [136]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel
def encode_token(unique_nodes):
  tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

  input_ids = tokenizer.encode(unique_nodes, return_tensors="pt")
  # input_ids = input_ids.transpose(0, 1)
  return input_ids[0]
# Now, 'word_embeddings' contains a list of word embeddings for each word in the word list





In [137]:
def batch_embeddings(contexts):
  max_seq_length = max(len(context) for context in contexts)

  # Pad sequences to the maximum length
  padded_embeddings = []
  for context in contexts:
      embeddings = word_embedding(context)
      pad_length = max_seq_length - len(context)
      padded_embeddings.append(np.pad(embeddings, ((0, pad_length), (0, 0)), mode='constant'))

  # Stack the padded embeddings to create a tensor
  batch_embeddings_tensor = torch.stack([torch.tensor(embedding) for embedding in padded_embeddings])
  return batch_embeddings_tensor

In [138]:
def get_weights(size, gain=1.414):
    weights = nn.Parameter(torch.zeros(size=size))
    nn.init.xavier_uniform_(weights, gain=gain)
    return weights


def get_bias(size):
    bias = nn.Parameter(torch.zeros(size=size))
    return bias


def get_act(act):
    if act.startswith('lrelu'):
        return nn.LeakyReLU(float(act.split(':')[1]))
    elif act == 'relu':
        return nn.ReLU()
    else:
        raise NotImplementedError

In [139]:
import torch
from torch_geometric.nn import GATv2Conv
import torch.nn.functional as F
from torch import nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the GAT model
class GATSelfAttention(nn.Module):
    def __init__(self, in_dim, out_dim, config, layer_id=0, head_id=0):
        """ One head GAT """
        super(GATSelfAttention, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.dropout = config.gnn_drop
        self.q_attn = config.q_attn
        self.query_dim = in_dim
        self.n_type = config.n_type

        # Case study
        self.layer_id = layer_id
        self.head_id = head_id
        self.step = 0

        self.W_type = nn.ParameterList()
        self.a_type = nn.ParameterList()
        self.qattn_W1 = nn.ParameterList()
        self.qattn_W2 = nn.ParameterList()
        for i in range(self.n_type):
            self.W_type.append(get_weights((in_dim, out_dim)))
            self.a_type.append(get_weights((out_dim * 2, 1)))

            if config.q_attn:
                q_dim = config.hidden_dim if config.q_update else config.input_dim
                self.qattn_W1.append(get_weights((q_dim, out_dim * 2)))
                self.qattn_W2.append(get_weights((out_dim * 2, out_dim * 2)))

        self.act = get_act('lrelu:0.2')

    def forward(self, input_state, adj, entity_mask, adj_mask=None, query_vec=None):

        zero_vec = torch.zeros_like(adj)
        scores = 0

        for i in range(self.n_type):
            # h = torch.matmul(input_state, self.W_type[i])
            h = torch.matmul(input_state.to(self.W_type[i].dtype), self.W_type[i])

            h = F.dropout(h, self.dropout, self.training)
            N, E, d = h.shape

            a_input = torch.cat([h.repeat(1, 1, E).view(N, E * E, -1), h.repeat(1, E, 1)], dim=-1)
            a_input = a_input.view(-1, E, E, 2*d)

            if self.q_attn:
                q_gate = F.relu(torch.matmul(query_vec.to(self.qattn_W1[i].dtype), self.qattn_W1[i]))
                q_gate = torch.sigmoid(torch.matmul(q_gate, self.qattn_W2[i]))
                a_input = a_input * q_gate[:, None, None, :]
                score = self.act(torch.matmul(a_input, self.a_type[i]).squeeze(3))
            else:
                score = self.act(torch.matmul(a_input, self.a_type[i]).squeeze(3))
            scores += torch.where(adj == i+1, score, zero_vec)

        zero_vec = -9e15 * torch.ones_like(scores)
        scores = torch.where(adj > 0, scores, zero_vec)

        # Ahead Alloc
        if adj_mask is not None:
            h = h * adj_mask

        coefs = F.softmax(scores, dim=2)  # N * E * E
        # h = coefs.unsqueeze(3) * h.unsqueeze(2)  # N * E * E * d
        # h = torch.sum(h, dim=1)
        return coefs


# Load the pre-trained BART model and tokenizer

# Define the GAT-BART Decoder
class GAT_BART_Decoder(torch.nn.Module):
    def __init__(self):
        super(GAT_BART_Decoder, self).__init__()

        model_name = "gpt2"  # Replace with the appropriate model name
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        # self.bart_model = bart_model
        # self.bart_tokenizer = bart_tokenizer

    def forward(self, encoder_output ,entitys, contexts, answers):

      questions = []
      i=0
      for context, answer,entity in zip(contexts, answers,entitys):


          # Combine the context and answer into a single input
          coefs=encoder_output[i]

          # h = torch.sum(coefs * entity.unsqueeze(2), dim=2)  # N * E * d
          h=(torch.matmul(coefs.float() ,entity.float())).unsqueeze(0)

          input_text = f"\n{context}\n {answer}\n"
          # input_text = f"{answer}\nQuestion:"
          # input_text = f"Answer: {answer}\nQuestion:"
          # Tokenize the input text

          # Tokenize the input text
          input_ids = self.tokenizer.encode(input_text, return_tensors="pt")

          new_input = torch.cat((h,input_ids), dim=1).long()
          # Generate questions using the model
          with torch.no_grad():
              output = self.model.generate(new_input, max_length=20, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95,length_penalty=0.8 )

          # Decode and append the generated question
          generated_question = self.tokenizer.decode(output[0], skip_special_tokens=True)
          questions.append(generated_question)
          i+=1
      return questions

# Initialize the GAT encoder and the GAT_BART_Decoder



In [140]:
import torch
from torch.utils.data import Dataset
import numpy as np
import ast

class CustomGraphDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
        self.max_size ,self.max_answer_size= self.find_max_x_size()

        query_max_size=self.max_answer_size

        # Call the method to find the maximum x
        # self.max_edge_size=self.find_max_edge_size()
        # print("self.max_x_size ",self.max_x_size )
        # print("self.max_edge_size",self.max_edge_size)
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):

        # Get the Data object at the specified index
        data = self.data_list[idx]
        # batch_data, batch_contexts, batch_answers, batch_questions, batch_unique_nodes = data
        batch_edge, batch_unique_nodes, batch_contexts, batch_answers, batch_questions = data
        # Extract the 'x' and 'edge_index' attributes from batch_data


        # Pad x with the maximum x value
        num_nodes =len(batch_unique_nodes)

        if num_nodes < self.max_size:
            padding_size = self.max_size - num_nodes
            x_padded = batch_unique_nodes + ['<PAD>'] * (padding_size)
        else:
            x_padded = batch_unique_nodes
        # print("x_padded",x_padded)

        entity = encode_token(x_padded)


        # print("entity",type(entity),entity)
        # x=(word_embedding(x_padded)).unsqueeze(1)
        # print(type(x))
        x=(word_embedding(x_padded))

        # print("here ",len(x_padded))

        num_edges = batch_edge.size(1)
        if num_edges < self.max_size:
           padding_length = max(0, self.max_size - num_edges)
           edge_index_padded = torch.nn.functional.pad(batch_edge, (0, padding_length, 0, padding_length))

        else:
            edge_index_padded = batch_edge

        # batch_answers= batch_answers.split()

        # num_answer=len(batch_answers)
        # if num_answer<self.max_answer_size:
        #     padding_answer_size = self.max_answer_size - num_answer
        #     answer_padded = batch_answers + [' '] * (padding_answer_size)
        # else:
        #     answer_padded = batch_answers
        padded_answer = batch_answers.ljust(self.max_answer_size)

        # answer_padded=word_embedding(answer_padded)

        # Return the data in a format suitable for DataLoader





        return edge_index_padded,x,entity,batch_contexts,batch_answers,batch_questions

    def find_max_x_size(self):
        max_x_size = 0
        max_answer_size=0
        for data in self.data_list:
            edge, batch_unique_nodes,context, answer, _ = data

            num_nodes = len(batch_unique_nodes)
            if num_nodes > max_x_size:

              max_x_size = max(max_x_size, num_nodes)
            answer_size=len(answer)

            if answer_size > max_answer_size:

                  max_answer_size = max(max_answer_size, answer_size)

        return max_x_size,max_answer_size



    def get_max_x(self):
        return self.max_x


In [141]:
import torch
import pickle
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [142]:
with open('/content/drive/MyDrive/preprocess_v2/test_data0_1000.pkl', 'rb') as file:
    train_data = pickle.load(file)

In [143]:
print("khar")
from torch.utils.data import DataLoader
from torch_geometric.data import Data, Batch
shuffle = True  # Shuffle the data for randomness
data=CustomGraphDataset(train_data)
print("ok")
# dataloader = DataLoader(data, batch_size=batch_size, shuffle=shuffle)


khar
ok


In [144]:
query_max_size=data.max_answer_size

In [145]:


# import torch.nn as nn
# def compute_loss(outputs, targets):
#     criteria = nn.CrossEntropyLoss()

#     # Initialize target to target['input_ids']
#     for (output, target) in zip(outputs,targets)

#         # Get the maximum sequence length in the batch
#         max_seq_length = max(len(output[0]), len(target[0]))

#         # Pad the output and target sequences to the maximum length
#         # output = torch.nn.functional.pad(output, (0, max_seq_length - len(output[0]))
#         output = torch.nn.functional.pad(output, (0, max_seq_length - len(output[0])))

#         target = torch.nn.functional.pad(target, (0, max_seq_length - len(target[0])))

#         loss = criteria(target.float(), output.float())
#         torch.set_grad_enabled(True)
#         loss.requires_grad = True
#     return loss
import torch.nn as nn

def compute_loss(outputs, targets):
    criteria = nn.CrossEntropyLoss()

    # Initialize a list to store individual losses for each pair of output and target
    losses = []

    # Iterate over each pair of output and target
    for output, target in zip(outputs, targets):

        target=encode_token(target)
        output=encode_token(output)
        # print("output",output)

        # print("target",target)
        # Get the maximum sequence length in the batch
        max_seq_length = max(len(output), len(target))
        # print("max_seq_length",max_seq_length)

        # Pad the output and target sequences to the maximum length
        output = torch.nn.functional.pad(output, (0, max_seq_length - len(output)))
        target = torch.nn.functional.pad(target, (0, max_seq_length - len(target)))

        loss = criteria(target.float(), output.float())
        losses.append(loss)

    # Compute the mean loss across all pairs
    mean_loss = sum(losses) / len(losses)

    return mean_loss


def compute_accuracy(output, target):
    # Ensure both sequences have the same length
    target = target['input_ids']

    # Get the maximum sequence length in the batch
    max_seq_length = max(len(output[0]), len(target[0]))

    # Pad the output and target sequences to the maximum length
    # output = torch.nn.functional.pad(output, (0, max_seq_length - len(output[0]))
    output = torch.nn.functional.pad(output, (0, max_seq_length - len(output[0])))

    target = torch.nn.functional.pad(target, (0, max_seq_length - len(target[0])))

    # Count the number of correct matches
    correct_matches = sum(1 for p, t in zip(output, target) if p == t)

    # Calculate accuracy as the ratio of correct matches to the total number of elements
    accuracy = correct_matches / len(output)

    return accuracy


In [146]:
import torch
import torch.nn as nn
from transformers import BartForConditionalGeneration
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel

class CustomFineTuneModel(nn.Module):
    def __init__(self, gat_encoder, bart_decoder):
        super(CustomFineTuneModel, self).__init__()
        self.gat_encoder = gat_encoder
        self.bart_decoder = bart_decoder

    def forward(self, edge, index,entity, batch_contexts, batch_answers, batch_questions):
        tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

        L=index.size(1)
        N=batch_size
        entity_mask = torch.tensor([[random.choice([0, 1]) for _ in range(L)] for _ in range(N)], dtype=torch.float32)

        adj_mask= None
        # print("sam")
        # print("index",index.size())
        # print("edge",edge.size())
        # print("query",batch_answers)
        query=[]
        for i in batch_answers:
           query_vec= tokenizer.encode(i, return_tensors="pt")
           query_vec = F.pad(query_vec, (0, query_max_size - query_vec.size(1)))
           query.append(query_vec)
          #  print(query_vec)
        result_query = torch.cat(query, dim=0)

        # print("query_vec ", result_query.size())
        encoder_output = self.gat_encoder(index, edge, entity_mask, adj_mask, result_query)

        # print("encoder_output",encoder_output)
        # print("entity",entity)
        output=self.bart_decoder(encoder_output, entity,batch_contexts, batch_answers)


        return output


In [147]:
from types import SimpleNamespace

config = SimpleNamespace(
    gnn_drop=0.5,
    q_attn=True,
    hidden_dim=8,
    n_type=4,
    q_update=False,
    input_dim= query_max_size #answer dimention
)
in_dim = 300  # Input feature dimension
out_dim = 300 # Output feature dimension
gat_encoder =  GATSelfAttention(in_dim, out_dim, config, layer_id=0, head_id=2)
gat_bart_decoder = GAT_BART_Decoder()

In [148]:
CustomFineTuneModel = CustomFineTuneModel(gat_encoder, gat_bart_decoder)


In [150]:
import torch
import pickle
from torch.utils.data import DataLoader
from torch_geometric.data import Data, Batch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define fine-tuning hyperparameters and optimizer (move optimizer definition here)
learning_rate = 0.001


# Instantiate the optimizer with the model's parameters
optimizer = torch.optim.Adam(CustomFineTuneModel.parameters(), lr=learning_rate)
CustomFineTuneModel = CustomFineTuneModel.to(device)

# Initialize variables to keep track of the best model and lowest loss
best_loss = float('inf')
best_model_state_dict = None



custom_dataset = CustomGraphDataset(train_data)


train_loader = DataLoader(custom_dataset, batch_size=10, shuffle=True)

# print("number of training data:",len(train_loader))
# print("number of validation data:",len(val_loader))

# print(len(train_loader))
# Fine-tune the model
for epoch in range(10):
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    # Training phase
    CustomFineTuneModel.train()

    for batch in train_loader:
        # if    num_batches == 2:
        #   break

        edge_index_padded,x_padded,entity,batch_contexts,batch_answers,batch_questions = batch
        # print("entity 1",entity)
        # print(edge_index_padded.size())
        # print(x_padded.size())


        # batch_data_x_edge= edge_index_padded.to(device)
        # batch_data_x_index= x_padded.to(device)
        # batch_contexts = batch_contexts.to(device)
        # batch_answers = batch_answers.to(device)
        # encode_question = batch_questions.to(device)

        output = CustomFineTuneModel(edge_index_padded,x_padded, entity,batch_contexts, batch_answers, batch_questions)

        loss = compute_loss(batch_questions, output)
        total_loss += loss.item()
        # Compute accuracy
        # accuracy = compute_accuracy(output, encode_question)
        # total_accuracy += accuracy

        num_batches += 1
        print("in the batch:",num_batches,"loss :",loss)
        loss=loss.requires_grad_(True)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate average loss and accuracy for the epoch
    average_loss = total_loss / num_batches
    # average_accuracy = total_accuracy / num_batches

    # Print training loss and accuracy for the epoch
    print(f'Training - Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}')
    model_state_dict = CustomFineTuneModel.state_dict()
    torch.save(best_model_state_dict, 'fine_tuned_model.pth')




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

in the batch: 1 loss : tensor(1.5137e+09)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

in the batch: 2 loss : tensor(2.0481e+09)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

in the batch: 18 loss : tensor(1.0935e+09)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

in the batch: 19 loss : tensor(1.8895e+09)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

in the batch: 20 loss : tensor(9.5975e+08)
Training - Epoch [10/10] - Loss: 1628729721.6000
